### differents $B_\theta$, petits $\theta$, differents $B_{sf}$

In [1]:
import os

In [2]:
%mkdir -p tmp

In [3]:
data_folder = os.path.join('..', 'Psychopy_data', 'pilot', 'B_theta_B_sf')#,'json')
print('data_folder =', data_folder)

data_folder = ../Psychopy_data/pilot/B_theta_B_sf


In [4]:
%mkdir -p {data_folder}

In [5]:
%%writefile tmp/exp.py
import sys
import numpy as np
import os
import imageio
import sys

data_folder = sys.argv[1]
print('\n Initializing')
N_B_theta = 3
N_B_sf = 2
B_thetas = np.pi*np.logspace(-5, -1.5, N_B_theta, base=2)
print('B_thetas = ', B_thetas)
B_sfs = np.logspace(-4, 0., N_B_sf, base=2)
print('B_sfs = ', B_sfs)
theta_max = np.pi/8

N_trial = 4
N_total_trials = N_trial * N_B_theta * N_B_sf      # number of trials
fixation_length = 0.25  # length of fixation, in second
stim_length = 0.25   # length of stim, in second

seed = 2020
np.random.seed(seed)
#thetas = (2*np.random.rand(N_total_trials)-1)*theta_max
#thetas = (2*np.random.rand(N_trial)-1)*theta_max
thetas = np.linspace(-theta_max, theta_max, N_trial)
thetas = np.ravel(thetas[None, :] * np.ones((N_B_theta * N_B_sf, 1)))
conditions = np.ravel(np.arange(N_B_theta * N_B_sf)[:, None] * np.ones(N_trial)).astype(np.int)
ind = np.random.permutation(np.arange(N_total_trials))
thetas = thetas[ind]
conditions = conditions[ind]
#np.random.shuffle(conditions)
i_B_thetas, i_B_sfs = conditions // N_B_theta, conditions % N_B_theta

# generate all clouds
#help(mc.envelope_gabor)
def generate_random_cloud(theta, B_theta, B_sf, seed, downscale = 1):
    fname = 'tmp/%s_%s_%s.png' % (theta, B_theta, B_sf)
    if not os.path.isfile(fname):
        import MotionClouds as mc
        fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)
        mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                                 B_V=0, theta=np.pi/2-theta, B_theta=B_theta)
        im = mc.random_cloud(mc_i, seed=seed)
        im = (mc.rectif(im) * 255).astype('uint8')
        imageio.imwrite(fname, im[:, :, 0])
    return fname

for i_trial in range(N_total_trials):
    MC = generate_random_cloud(thetas[i_trial], 
                               B_theta=B_thetas[i_B_thetas[i_trial]], 
                               B_sf=B_sfs[i_B_sfs[i_trial]], 
                               seed=seed+i_trial) 

if len(sys.argv)>2:
    exp_observer = sys.argv[2]

    exp_name = "Perception de l'orientation"
    
    fullscr = False
    fullscr = True


    from psychopy import visual #, core, event, gui, data
    win = visual.Window([1400, 800], fullscr=fullscr, color=[0, 0, 0])


    mouse = event.Mouse(newPos=(0, 0), visible=False)
    x0, y0 = mouse.getPos()
    event.clearEvents()
    instructions = """
    At each trial you see one textured image, report the 
    orientation you perceive.

    Move the mouse to the right if it is clock-wise of the vertical,
    move the mouse to the left if it is counter clock-wise of the vertical.

    Move the mouse to begin the experiment.
    """
    msg = visual.TextStim(win, text=instructions, alignHoriz='center', alignVert='center', color='black')
    msg.draw()
    win.flip()
    while not mouse.mouseMoved(distance=.05): 
        event.clearEvents()
    fixation = visual.ShapeStim(vertices='cross', win=win, size=0.05, pos=[0, 0], fillColor='black', units='height')
    fixation.autolog = False

    bitmap = visual.ImageStim(win, MC, mask='gauss', size=0.8, units='height', interpolate=True)
    bitmap.autolog = False 

    msg = visual.TextStim(win, text='.', color='black')

    import time
    start_time = time.time()
    print("Test started")

    ans_list = []

    # run experiment
    for i_trial in range(N_total_trials):
        clock = core.Clock()
        fixation.draw()
        win.flip()

        MC = generate_random_cloud(thetas[i_trial], 
                           B_theta=B_thetas[i_B_thetas[i_trial]], 
                           B_sf=B_sfs[i_B_sfs[i_trial]], 
                           seed=seed+i_trial)
        bitmap.setImage(MC)

         # Times the trial
        while clock.getTime() < fixation_length + stim_length:
            if clock.getTime() < fixation_length:  # fixation
                fixation.draw()
            elif clock.getTime() < fixation_length + stim_length: 
                bitmap.draw()
            win.flip()

        msg.draw()
        win.flip()

        mouse.setPos(newPos=(0, 0))
        x0, y0 = mouse.getPos()
        event.clearEvents()

        while not mouse.mouseMoved(distance=.025): 
            # this creates a never-ending loop
            # until we move the mouse
            event.clearEvents()

        RT = clock.getTime()

        x, y = mouse.getPos()
        if x < 0:
            answer = 'left'
        else: #elif x > 0:
            answer = 'right'
            if x == 0: print('this should not happen 😤')
        event.clearEvents() 

        ans = event.getKeys()       

        if ans == ['escape', 'q']:
            win.close()
            core.quit()

        correct = ((thetas[i_trial] > 0) and answer == 'right') or ((thetas[i_trial] < 0) and answer == 'left')
        print('At trial ', i_trial, 'Angle=', '%3.3f' % (thetas[i_trial]*180/np.pi), 'answer is ',  answer, '(correct=', correct, ');')   

        ans_list.append([i_trial, thetas[i_trial], answer, 
                         B_thetas[i_B_thetas[i_trial]], B_sfs[i_B_sfs[i_trial]], 
                         i_B_thetas[i_trial], i_B_sfs[i_trial], RT])

    print('saving')
    import json
    print(ans_list)
    json.dump(ans_list, open(os.path.join(data_folder, f'Psychophys_discrim_{exp_observer}.json'), 'w', encoding="utf8"))

    print('exiting')
    event.clearEvents() 
    win.close()
    core.quit()


Overwriting tmp/exp.py


In [6]:
import numpy as np
N_B_theta = 3
N_B_sf = 5

seed = 42
np.random.seed(seed)
N_trial = 4
N_total_trials = N_trial * N_B_theta * N_B_sf      # number of trials

theta_max = 20
thetas = np.linspace(-theta_max, theta_max, N_trial)
# thetas = np.ravel((2*np.random.rand(N_trial, 1)-1)*20 * np.ones(N_B_theta * N_B_sf))
thetas = np.ravel(thetas[None, :] * np.ones((N_B_theta * N_B_sf, 1)))
thetas

array([-20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ,
       -20.        ,  -6.66666667,   6.66666667,  20.        ])

In [7]:
conditions = np.ravel(np.arange(N_B_theta * N_B_sf)[:, None] * np.ones(N_trial)).astype(np.int)
conditions

array([ 0,  0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  4,
        4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  8,
        8,  8,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12,
       12, 13, 13, 13, 13, 14, 14, 14, 14])

In [8]:
conditions // N_B_theta, conditions % N_B_theta, (conditions // N_B_theta)*N_B_theta + conditions % N_B_theta

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2,
        2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]),
 array([ 0,  0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  4,
         4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  8,
         8,  8,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12,
        12, 13, 13, 13, 13, 14, 14, 14, 14]))

In [9]:
#np.random.shuffle(conditions)
conditions

array([ 0,  0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  4,
        4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  8,
        8,  8,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12,
       12, 13, 13, 13, 13, 14, 14, 14, 14])

In [10]:
ind = np.random.permutation(np.arange(N_B_theta * N_B_sf*N_trial))
conditions = conditions[ind]
ind, conditions, thetas[ind]

(array([ 0,  5, 36, 45, 13, 54, 33, 48, 12, 57, 46, 50, 31,  3, 52, 17,  8,
         6, 40,  4, 43, 19, 34, 58, 25, 56, 15, 27,  9, 30, 26, 16, 24, 55,
        11, 32, 53, 41, 37, 29, 44,  1, 21,  2, 47, 39, 35, 23, 49, 10, 22,
        18, 59, 20,  7, 42, 14, 28, 51, 38]),
 array([ 0,  1,  9, 11,  3, 13,  8, 12,  3, 14, 11, 12,  7,  0, 13,  4,  2,
         1, 10,  1, 10,  4,  8, 14,  6, 14,  3,  6,  2,  7,  6,  4,  6, 13,
         2,  8, 13, 10,  9,  7, 11,  0,  5,  0, 11,  9,  8,  5, 12,  2,  5,
         4, 14,  5,  1, 10,  3,  7, 12,  9]),
 array([-20.        ,  -6.66666667, -20.        ,  -6.66666667,
         -6.66666667,   6.66666667,  -6.66666667, -20.        ,
        -20.        ,  -6.66666667,   6.66666667,   6.66666667,
         20.        ,  20.        , -20.        ,  -6.66666667,
        -20.        ,   6.66666667, -20.        , -20.        ,
         20.        ,  20.        ,   6.66666667,   6.66666667,
         -6.66666667, -20.        ,  20.        ,  20.        ,
    

In [11]:
conditions // N_B_theta, conditions % N_B_theta

(array([0, 0, 3, 3, 1, 4, 2, 4, 1, 4, 3, 4, 2, 0, 4, 1, 0, 0, 3, 0, 3, 1,
        2, 4, 2, 4, 1, 2, 0, 2, 2, 1, 2, 4, 0, 2, 4, 3, 3, 2, 3, 0, 1, 0,
        3, 3, 2, 1, 4, 0, 1, 1, 4, 1, 0, 3, 1, 2, 4, 3]),
 array([0, 1, 0, 2, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1,
        2, 2, 0, 2, 0, 0, 2, 1, 0, 1, 0, 1, 2, 2, 1, 1, 0, 1, 2, 0, 2, 0,
        2, 0, 2, 2, 0, 2, 2, 1, 2, 2, 1, 1, 0, 1, 0, 0]))

In [12]:
(conditions // N_B_theta)*N_B_theta + conditions % N_B_theta

array([ 0,  1,  9, 11,  3, 13,  8, 12,  3, 14, 11, 12,  7,  0, 13,  4,  2,
        1, 10,  1, 10,  4,  8, 14,  6, 14,  3,  6,  2,  7,  6,  4,  6, 13,
        2,  8, 13, 10,  9,  7, 11,  0,  5,  0, 11,  9,  8,  5, 12,  2,  5,
        4, 14,  5,  1, 10,  3,  7, 12,  9])

In [13]:
!ipython tmp/exp.py {data_folder} 

]0;IPython: Notebooks/W17
 Initializing
B_thetas =  [0.09817477 0.18005332 0.33021925 0.60562477 1.11072073]
B_sfs =  [0.0625 0.125  0.25   0.5    1.    ]


In [18]:
%%writefile tmp/test.py

from psychopy import visual, core
import time
fullscr = False
#win = visual.Window([1400,800],fullscr=fullscr, color=[0,0,0], winType='pygame', monitor='testMonitor', units='height')
win = visual.Window([1400, 800], fullscr=fullscr, color=[0, 0, 0])
msg = visual.TextStim(win, text='.', color='black')
msg.draw()
win.flip()
time.sleep(2)
msg.setText('toto')
msg.draw()
win.flip()
time.sleep(2)
print('exiting')
win.close()
core.quit()


Overwriting tmp/test.py


In [19]:
!ipython tmp/test.py 

]0;IPython: Notebooks/W17pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
2020-05-05 15:14:17.780 Python[8692:1870061] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to (null)
exiting
2.2523 	WARNING 	Monitor specification not found. Creating a temporary one...
5.1723 	WARNING 	Couldn't measure a consistent frame rate.
  - Is your graphics card set to sync to vertical blank?
  - Are you running other processes on your computer?

5.2354 	WARNING 	t of last frame was 63.36ms (=1/15)
7.2451 	WARNING 	t of last frame was 2009.72ms (=1/0)
